In [1]:
# load data
!wget https://archive.org/download/nf_prize_dataset.tar/nf_prize_dataset.tar.gz
!tar -xzf nf_prize_dataset.tar.gz
!tar -xf download/training_set.tar

--2021-11-10 00:45:42--  https://archive.org/download/nf_prize_dataset.tar/nf_prize_dataset.tar.gz
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia600205.us.archive.org/7/items/nf_prize_dataset.tar/nf_prize_dataset.tar.gz [following]
--2021-11-10 00:45:43--  https://ia600205.us.archive.org/7/items/nf_prize_dataset.tar/nf_prize_dataset.tar.gz
Resolving ia600205.us.archive.org (ia600205.us.archive.org)... 207.241.227.225
Connecting to ia600205.us.archive.org (ia600205.us.archive.org)|207.241.227.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 697552028 (665M) [application/octet-stream]
Saving to: ‘nf_prize_dataset.tar.gz’

nf_prize_dataset.ta 100%[===================>] 665,24M   672KB/s    in 44m 49s 

2021-11-10 01:30:33 (253 KB/s) - ‘nf_prize_dataset.tar.gz’ saved [697552028/697552028]



In [1]:
# data preprocessing
import os
import numpy as np
import math
from tqdm import trange

entries = np.zeros((100480507, 3), dtype = int)

training_set = 'training_set/'
i = 0

for filename in os.listdir(training_set):
    file = training_set + filename
    with open(file) as f:
        lines = f.readlines()
        movie_id = int(lines[0].split(':')[0])
        for line in lines[1:]:
            user_id, user_score, date = line.split(',')
            user_id, user_score = int(user_id), int(user_score)
            entries[i] = movie_id, user_id, user_score
            i+=1

# shuffle 
entries = entries[np.random.permutation(len(entries))]
entries[:5]

array([[  14759, 1124804,       4],
       [   3579,  415594,       4],
       [  10491, 1616888,       5],
       [  13255,  137051,       4],
       [  14574,  960577,       4]])

In [9]:
# data processing
user_ids = {x:i for i,x in enumerate(set(entries[:, 1]))} # create user ids list

movies_number = 17770
users_number = len(user_ids)

x = np.zeros((entries.shape[0], 2))

# movie id, translate to [0, MOVIES_NUM-1]
x[:, 0] = entries[:, 0] - 1

# user id, translate to [0, size NUMBER_OF_USERS-1], offset by MOVIES_NUM
x[:, 1] = movies_number + np.vectorize(user_ids.get)(entries[:, 1])

# for indexing
x = x.astype(int)

# rating
y = entries[:, 2]  

In [10]:
# k-fold cross validation

class KFoldsValidation:
    def __init__(self, x, y, folds_num = 5):
        fold_range = range(folds_num)
        self.folds_num = folds_num
        self.x_folds = [x[i::folds_num] for i in fold_range]
        self.y_folds = [y[i::folds_num] for i in fold_range]
        
    def get(self, fold_i):
        fold_range = range(self.folds_num)
        
        x_fold = self.x_folds[fold_i]
        y_fold = self.y_folds[fold_i]
        
        # remove current fold from train set        
        x_train = np.concatenate([self.x_folds[i] for i in fold_range if i != fold_i])
        y_train = np.concatenate([self.y_folds[i] for i in fold_range if i != fold_i])
        
        return x_train, y_train, x_fold, y_fold

In [11]:
def rmse_score(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))**0.5

def r2_score(y, y_pred):
    y_avg = y.mean()
    ss_total = np.sum(np.square(y - y_avg))
    ss_err = np.sum(np.square(y - y_pred))
    return (1 - ss_err/ss_total)


# reference for Adam: https://ruder.io/optimizing-gradient-descent/index.html#challenges
class FactorizationMachine: #todo
    def __init__(self, n, k):
            
        self.t = 0
        self.eps = 1e-8
        self.lr = 0.01
        
        # adam hyperparams, decay rates
        self.beta_1 = 0.9
        self.beta_2 = 0.999
        
        self.w_0 = 0
        self.w = 0.01 * np.random.randn(n)
        self.v = 0.01 * np.random.randn(n,k)

        # adam moments
        self.v_dw_0 = np.zeros_like(self.w_0)
        self.s_dw_0 = np.zeros_like(self.w_0)
        self.v_dw = np.zeros_like(self.w)
        self.s_dw = np.zeros_like(self.w)
        self.v_dv = np.zeros_like(self.v)
        self.s_dv = np.zeros_like(self.v)

        # cache to be used in backward pass
        self.x_batch = None
        self.v_dot_x = None

    def forward(self, x):

        n = total_features
        k = 3
        

        # cache
        self.x_batch = x
        self.v_dot_x = np.sum(self.v[x], axis=1)
        
        return self.w_0   \
               + np.sum(self.w[x], axis=1)   \
               + 0.5 * np.sum(np.square(self.v_dot_x) - np.sum(np.square(self.v[x]), axis=1), axis=1)
         
    def backward(self, dLdy):
        
        n = total_features
        k = 3
        
        if self.x_batch is None:
            assert 0, 'Call forward first'

        #Gradient w.r.t. bias
        dLdw_0 = np.mean(dLdy)

        #Gradient w.r.t. linear weights
        dLdw = np.zeros(n)
        for x, dLdyi in zip(self.x_batch, dLdy):
            dLdw[x] +=  dLdyi
        dLdw /= dLdy.shape[0]

        #Gradient w.r.t. pairwise weights
        dLdv = np.zeros((n,k))
        for x, v_dot_xi, dLdyi in zip(self.x_batch, self.v_dot_x, dLdy):
            dLdv[x] += dLdyi * (v_dot_xi - self.v[x])
        dLdv /= dLdy.shape[0]

        # estimate moments
        # compute the decaying averages of past and past squared gradients 
        self.v_dw_0 = self.beta_1 * self.v_dw_0 + (1 - self.beta_1) * dLdw_0
        self.s_dw_0 = self.beta_2 * self.s_dw_0 + (1 - self.beta_2) * dLdw_0 * dLdw_0
        
        self.v_dw = self.beta_1 * self.v_dw + (1 - self.beta_1) * dLdw
        self.s_dw = self.beta_2 * self.s_dw + (1 - self.beta_2) * dLdw * dLdw
        
        self.v_dv = self.beta_1 * self.v_dv + (1 - self.beta_1) * dLdv
        self.s_dv = self.beta_2 * self.s_dv + (1 - self.beta_2) * dLdv * dLdv
        
        # computing bias-corrected first and second moment estimates
        self.t += 1
        bias_correction_1 = 1 - self.beta_1**self.t
        bias_correction_2 = 1 - self.beta_2**self.t

        step_size = self.lr / bias_correction_1

        denom_dw_0 = np.sqrt(self.s_dw_0) / math.sqrt(bias_correction_2) + self.eps
        denom_dw = np.sqrt(self.s_dw) / math.sqrt(bias_correction_2) + self.eps
        denom_dv = np.sqrt(self.s_dv) / math.sqrt(bias_correction_2) + self.eps

        # Adam update rule
        self.w_0 -= step_size * self.v_dw_0 / denom_dw_0
        self.w -= step_size * self.v_dw / denom_dw
        self.v -= step_size * self.v_dv / denom_dv
        
        #Clear cache
        self.v_dot_x = None
        self.x_batch = None


In [12]:
# error estimation
class MSE:
    def __init__(self):
        self.error = None
        
    def forward(self, y_true, y_predicted):
        self.error = y_true - y_predicted
        return np.mean(np.square(self.error))
    
    def backward(self):
        if self.error is None:
            assert 0, 'Call forward first'
        return self.error * -2

In [13]:
total_features = users_number + movies_number

kfold = KFoldsValidation(x, y)

In [14]:
batch_size = 20000

def get_batch(x,y, i):
    return x[i * batch_size:(i + 1) * batch_size], y[i * batch_size:(i + 1) * batch_size]

In [15]:
from tqdm import trange


def compute_with_factorization_machine(epochs, n_movies, n_users, k):
    
    rmses = []
    r2s = []
    
    n = n_movies + n_users
    criterion = MSE()
    
    for fold_i in range(kfold.folds_num):
        model = FactorizationMachine(n,k)    
        X_train, y_train, X_test, y_test = kfold.get(fold_i)

        #TRAIN
        print(" Train on fold {}".format(fold_i+1))

        batch_size = 20000
        iters = X_train.shape[0] // batch_size
        if (X_train.shape[0] % batch_size > 0):
            iters += 1

        # for current task the number of epoches is 1    
        for epoch in range(epochs):
            print("Epoch", epoch + 1)
            running_loss = 0
            running_r2 = 0
            with trange(iters) as t:
                for i in t:
                    X_batch, y_batch = get_batch(X_train,y_train, i)

                    y_pred = model.forward(X_batch)
                    loss = criterion.forward(y_batch, y_pred)
                    dLdy = criterion.backward()
                    model.backward(dLdy)

                    running_loss += loss
                    r2 = r2_score(y_batch, y_pred)
                    running_r2 += r2

                    t.set_postfix(mse=loss, r2=r2)

            running_loss /= X_train.shape[0]/batch_size
            running_r2 /= X_train.shape[0]/batch_size
            print()
            print("MSE = {}, R2 = {}".format(running_loss, running_r2))

        #TEST
        print(" Test on {} fold".format(fold_i+1))

        batch_size = 20000
        iters = X_test.shape[0] // batch_size
        if (X_test.shape[0] % batch_size > 0):
            iters += 1

        y_pred = np.zeros(y_test.shape)
        for i in trange(iters):
            X_batch, _ = get_batch(X_test,y_test, i)
            y_pred[i*batch_size:(i+1)*batch_size] = model.forward(X_batch)

        print()
        rmses.append(rmse_score(y_test, y_pred))
        r2s.append(r2_score(y_test, y_pred))
        print("RMSE = {}, R2 = {}".format(rmses[-1], r2s[-1]))
        print()
    
    
if __name__ == '__main__':

    epochs = 1
    n_movies = 17770
    n_users = len(user_ids)
    k = 3
   
    compute_with_factorization_machine(epochs, n_movies, n_users, k)
    
    

  0%|          | 0/4020 [00:00<?, ?it/s]

 Train on fold 1
Epoch 1


  1%|▏         | 14/1005 [00:00<00:07, 131.12it/s]


MSE = 1.0476009468915124, R2 = 0.11059035189765729
 Test on 1 fold


100%|██████████| 1005/1005 [00:07<00:00, 133.92it/s]


...

 Train on fold 5
Epoch 1


  1%|▏         | 14/1005 [00:00<00:07, 131.77it/s]


MSE = 1.0527461040150123, R2 = 0.10615971659362171
 Test on 5 fold


100%|██████████| 1005/1005 [00:07<00:00, 134.17it/s]



RMSE = 0.8901148970730431, R2 = 0.32729048514378944

